In [ ]:
# 행렬 계산 및 데이터 프레임 다루기
import numpy as np
import pandas as pd
# 시각화용
import matplotlib.pyplot as plt
# 웹에 있 데이터 다루기
import urllib.request

# 데이터 셋 나누기(학습, 확인)
from sklearn.model_selection import train_test_split

# 단어(문자)를 수치화 및 패딩으로 자리 맞추기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin1')
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 5572


In [ ]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [ ]:
data.columns

del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

data['v1']=data['v1'].replace(['ham', 'spam'],[0,1])
data.columns=['label', 'title']
data.head(3)

,label,title
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [ ]:
data.info()
print()
data.describe()
print()
data.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   int64 
 1   title   5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB




label  title                                                                                                                                                                                                        
0      Sorry, I'll call later                                                                                                                                                                                           30
       I cant pick the phone right now. Pls send a message                                                                                                                                                              12
       Ok...                                                                                                                                                                                                            10
1      Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£

In [ ]:
# 중복되는 제목 확인
print(data['title'].nunique())

# title이 중복인 데이터 지우기
data.drop_duplicates(subset=['title'],inplace=True)
print(len(data['title']))

5169
5169


In [ ]:
ydata=data['label']
xdata=data['title']

In [ ]:
# 실습 data는 y값의 편차가 심해, train으로 다 몰릴 수 있다.
# 층화추출로 해결 가능 stratify
# 층화추출: 종류에따라 균등하게 분리하여 추출하는 방법
# random_state= 난수
xtrain, xtest, ytrain, ytest=train_test_split(xdata,ydata, test_size=0.2, random_state=902, stratify=ydata)
print(len(ytest))
print(xtrain.iloc[0])
print(xtest.iloc[0])

1034
I don,t think so. You don't need to be going out that late on a school night. ESPECIALLY when the one class you have is the one you missed last wednesday and probably failed a test in on friday
Hey... Why dont we just go watch x men and have lunch... Haha 


In [ ]:
xtrain

2178    I don,t think so. You don't need to be going o...
3320                               Yo im right by yo work
2546    Company is very good.environment is terrific a...
3033      Get ready for  &lt;#&gt;  inches of pleasure...
5114              Argh why the fuck is nobody in town ;_;
                              ...                        
2408            Dear where you will be when i reach there
5515    You are a great role model. You are giving so ...
1199    NEFT Transaction with reference number  &lt;#&...
5155    MY NEW YEARS EVE WAS OK. I WENT TO A PARTY WIT...
4287    Hey mr whats the name of that bill brison book...
Name: title, Length: 4135, dtype: object

In [ ]:
ytrain.value_counts()

0    3613
1     522
Name: label, dtype: int64

In [ ]:
list(xtrain.values)

["I don,t think so. You don't need to be going out that late on a school night. ESPECIALLY when the one class you have is the one you missed last wednesday and probably failed a test in on friday",
 'Yo im right by yo work',
 'Company is very good.environment is terrific and food is really nice:)',
 'Get ready for  &lt;#&gt;  inches of pleasure...',
 'Argh why the fuck is nobody in town ;_;',
 'Oh great. I.ll disturb him more so that we can talk.',
 "Who's there say hi to our drugdealer",
 'Me hungry buy some food good lei... But mum n yun dun wan juz buy a little bit... ',
 'Cancel cheyyamo?and get some money back?',
 'How much would it cost to hire a hitman',
 'Lol! Oops sorry! Have fun. ',
 "Alright we'll bring it to you, see you in like  &lt;#&gt;  mins",
 '<Forwarded from 21870000>Hi - this is your Mailbox Messaging SMS alert. You have 40 matches. Please call back on 09056242159 to retrieve your messages and matches cc100p/min',
 "So i asked how's anthony. Dad. And your bf",
 'Ind

In [ ]:
xtrain_list=[]
xtrain_list.extend(xtrain.values)
xtrain_list

["I don,t think so. You don't need to be going out that late on a school night. ESPECIALLY when the one class you have is the one you missed last wednesday and probably failed a test in on friday",
 'Yo im right by yo work',
 'Company is very good.environment is terrific and food is really nice:)',
 'Get ready for  &lt;#&gt;  inches of pleasure...',
 'Argh why the fuck is nobody in town ;_;',
 'Oh great. I.ll disturb him more so that we can talk.',
 "Who's there say hi to our drugdealer",
 'Me hungry buy some food good lei... But mum n yun dun wan juz buy a little bit... ',
 'Cancel cheyyamo?and get some money back?',
 'How much would it cost to hire a hitman',
 'Lol! Oops sorry! Have fun. ',
 "Alright we'll bring it to you, see you in like  &lt;#&gt;  mins",
 '<Forwarded from 21870000>Hi - this is your Mailbox Messaging SMS alert. You have 40 matches. Please call back on 09056242159 to retrieve your messages and matches cc100p/min',
 "So i asked how's anthony. Dad. And your bf",
 'Ind

In [ ]:
# 소문자 변환 및 punctuation
def pre_func(title):
    res=''.join(w.lower() for w in title if w not in punctuation)
    return res

pre_xtrain=[pre_func(x) for x in xtrain_list]
pre_xtrain[0:30]

['i dont think so you dont need to be going out that late on a school night especially when the one class you have is the one you missed last wednesday and probably failed a test in on friday',
 'yo im right by yo work',
 'company is very goodenvironment is terrific and food is really nice',
 'get ready for  ltgt  inches of pleasure',
 'argh why the fuck is nobody in town ',
 'oh great ill disturb him more so that we can talk',
 'whos there say hi to our drugdealer',
 'me hungry buy some food good lei but mum n yun dun wan juz buy a little bit ',
 'cancel cheyyamoand get some money back',
 'how much would it cost to hire a hitman',
 'lol oops sorry have fun ',
 'alright well bring it to you see you in like  ltgt  mins',
 'forwarded from 21870000hi  this is your mailbox messaging sms alert you have 40 matches please call back on 09056242159 to retrieve your messages and matches cc100pmin',
 'so i asked hows anthony dad and your bf',
 'indeed and by the way it was either or  not both ',


In [ ]:
# 불용어 제거 사전에 불용어 추가
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

xtrain2 = []

for sentence in pre_xtrain :
    xtrain2.append(' '.join([word for word in sentence.split(' ') if word not in stopwords.words('english')]))

print(xtrain2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['dont think dont need going late school night especially one class one missed last wednesday probably failed test friday', 'yo im right yo work', 'company goodenvironment terrific food really nice', 'get ready  ltgt  inches pleasure', 'argh fuck nobody town ', 'oh great ill disturb talk', 'whos say hi drugdealer', 'hungry buy food good lei mum n yun dun wan juz buy little bit ', 'cancel cheyyamoand get money back', 'much would cost hire hitman', 'lol oops sorry fun ', 'alright well bring see like  ltgt  mins', 'forwarded 21870000hi  mailbox messaging sms alert 40 matches please call back 09056242159 retrieve messages matches cc100pmin', 'asked hows anthony dad bf', 'indeed way either  ', 'norm tomorrow finish 415 cos st tests need sort library stuff point tomo  got letter today  access til end march better get move', 'indians r poor india poor country says one swiss bank directors says  ltgt  lac crore indian money deposited swiss banks used taxless budget  ltgt  yrs give  ltgt  crore

In [ ]:
tok=Tokenizer()
tok.fit_on_texts(xtrain2)
xtrain_enc=tok.texts_to_sequences(xtrain2)

In [ ]:
xtrain_enc
w2i=tok.word_index
vocab_size=len(w2i)+1

In [ ]:
maxLen=max(map(len, xtrain_enc))

In [ ]:
xtrain_padded=pad_sequences(xtrain_enc, maxlen=maxLen)

In [ ]:
xtrain_padded.shape

(4135, 80)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size, 32))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history=model.fit(xtrain_padded, ytrain, epochs=5, batch_size=32,
                  validation_split=0.2)

Epoch 1/5
104/104 [==============================] - 6s 41ms/step - loss: 0.4256 - accuracy: 0.8449 - val_loss: 0.3792 - val_accuracy: 0.8646
Epoch 2/5
104/104 [==============================] - 5s 44ms/step - loss: 0.2114 - accuracy: 0.9226 - val_loss: 0.1477 - val_accuracy: 0.9553
Epoch 3/5
104/104 [==============================] - 4s 37ms/step - loss: 0.0795 - accuracy: 0.9749 - val_loss: 0.2293 - val_accuracy: 0.9069
Epoch 4/5
104/104 [==============================] - 3s 30ms/step - loss: 0.0616 - accuracy: 0.9819 - val_loss: 0.0875 - val_accuracy: 0.9746
Epoch 5/5
104/104 [==============================] - 4s 40ms/step - loss: 0.0439 - accuracy: 0.9840 - val_loss: 0.0942 - val_accuracy: 0.9698


In [ ]:
# xtest_enc=tok.texts_to_sequences(xtest)
xtest_enc=tok.texts_to_sequences(xtest)
xtest_padded=pad_sequences(xtest_enc, maxlen=maxLen)
# model.predict(xtest_padded)
model.evaluate(xtest_padded, ytest)

33/33 [==============================] - 0s 8ms/step - loss: 0.0922 - accuracy: 0.9729


[0.09216941148042679, 0.9729207158088684]